In [1]:
# Libraries

import numpy as np
from statsmodels.tsa.arima_model import ARIMA
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from numpy import log


In [2]:
%scala 
val driver = "org.postgresql.Driver"
val jdbcUsername = "data_sci"
val jdbcPassword = "dataSci4lyf"



val jdbcUrl = "jdbc:postgresql://cpdb-databricks.cgod7egsd6vr.us-east-2.rds.amazonaws.com/cpdb"

import java.util.Properties
val connectionProperties = new Properties()

connectionProperties.put("user", jdbcUsername)
connectionProperties.put("password", jdbcPassword)
connectionProperties.setProperty("Driver", driver)

driver: String = org.postgresql.Driver
jdbcUsername: String = data_sci
jdbcPassword: String = dataSci4lyf
jdbcUrl: String = jdbc:postgresql://cpdb-databricks.cgod7egsd6vr.us-east-2.rds.amazonaws.com/cpdb
import java.util.Properties
connectionProperties: java.util.Properties = {user=data_sci, password=dataSci4lyf, Driver=org.postgresql.Driver}
res5: Object = null

In [3]:
%scala 
val data_officer_sql = "(select * from data_officer) doc_tags"
val data_officer = spark.read.jdbc(url=jdbcUrl, table=data_officer_sql, properties=connectionProperties)
data_officer.createOrReplaceTempView("data_officer")

val data_officerallegation_sql = "(select * from data_officerallegation) doc_tags"
val data_officerallegation = spark.read.jdbc(url=jdbcUrl, table=data_officerallegation_sql, properties=connectionProperties)
data_officerallegation.createOrReplaceTempView("data_officerallegation")

val data_allegation_sql = "(select * from data_allegation) doc_tags"
val data_allegation = spark.read.jdbc(url=jdbcUrl, table=data_allegation_sql, properties=connectionProperties)
data_allegation.createOrReplaceTempView("data_allegation")

data_officer_sql: String = (select * from data_officer) doc_tags
data_officer: org.apache.spark.sql.DataFrame = [id: int, gender: string ... 17 more fields]
data_officerallegation_sql: String = (select * from data_officerallegation) doc_tags
data_officerallegation: org.apache.spark.sql.DataFrame = [id: int, start_date: date ... 11 more fields]
data_allegation_sql: String = (select * from data_allegation) doc_tags
data_allegation: org.apache.spark.sql.DataFrame = [id: int, crid: string ... 11 more fields]

In [4]:


# Create training, testing and predict sets

data_training = spark.sql('select cast(CONCAT(extract(year from data_allegation.incident_date), extract(month from data_allegation.incident_date) + 10) as int) as id_, count(data_officerallegation.id) as count_, CONCAT(extract(month from data_allegation.incident_date), "/01/", extract(year from data_allegation.incident_date)) as date from data_officerallegation inner join data_allegation on data_officerallegation.allegation_id = data_allegation.id where data_allegation.incident_date is not null group by id_, date having id_ >= 200000 and id_ < 201400 order by id_ ').toPandas() 

data_training = data_training.set_index('date')

data_testing = spark.sql('select cast(CONCAT(extract(year from data_allegation.incident_date), extract(month from data_allegation.incident_date) + 10) as int) as id_, count(data_officerallegation.id) as count_, CONCAT(extract(month from data_allegation.incident_date), "/01/", extract(year from data_allegation.incident_date)) as date from data_officerallegation inner join data_allegation on data_officerallegation.allegation_id = data_allegation.id where data_allegation.incident_date is not null group by id_, date having id_ >= 201400 and id_ < 201500 order by id_').toPandas()
 
data_testing = data_testing.set_index('date')

data_predict = spark.sql('select cast(CONCAT(extract(year from data_allegation.incident_date), extract(month from data_allegation.incident_date) + 10) as int) as id_, count(data_officerallegation.id) as count_, CONCAT(extract(month from data_allegation.incident_date), "/01/", extract(year from data_allegation.incident_date)) as date from data_officerallegation inner join data_allegation on data_officerallegation.allegation_id = data_allegation.id where data_allegation.incident_date is not null group by id_, date having id_ >= 201500 order by id_').toPandas()

data_predict = data_predict.set_index('date')


/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [5]:
# Drop columns which are not needed
data_training = data_training.drop(['id_'], axis=1)
data_testing = data_testing.drop(['id_'], axis=1)
data_predict = data_predict.drop(['id_'], axis=1)
print(data_training)



count_
date 
1/01/2000 1710
2/01/2000 1670
3/01/2000 1753
4/01/2000 1672
5/01/2000 1519
6/01/2000 1700
7/01/2000 1758
8/01/2000 1973
9/01/2000 1586
10/01/2000 1680
11/01/2000 1276
12/01/2000 1143
1/01/2001 1380
2/01/2001 1009
3/01/2001 1202
4/01/2001 1069
5/01/2001 952
6/01/2001 1045
7/01/2001 1072
8/01/2001 1046
9/01/2001 927
10/01/2001 865
11/01/2001 966
12/01/2001 789
1/01/2002 964
2/01/2002 839
3/01/2002 937
4/01/2002 938
5/01/2002 1060
6/01/2002 1085
... ...
7/01/2011 525
8/01/2011 597
9/01/2011 514
10/01/2011 570
11/01/2011 462
12/01/2011 417
1/01/2012 383
2/01/2012 422
3/01/2012 479
4/01/2012 505
5/01/2012 483
6/01/2012 546
7/01/2012 530
8/01/2012 559
9/01/2012 445
10/01/2012 429
11/01/2012 345
12/01/2012 403
1/01/2013 430
2/01/2013 349
3/01/2013 457
4/01/2013 556
5/01/2013 529
6/01/2013 419
7/01/2013 445
8/01/2013 500
9/01/2013 423
10/01/2013 397
11/01/2013 446
12/01/2013 301

[168 rows x 1 columns]

In [6]:
print(data_training.index)

Index(['1/01/2000', '2/01/2000', '3/01/2000', '4/01/2000', '5/01/2000',
 '6/01/2000', '7/01/2000', '8/01/2000', '9/01/2000', '10/01/2000',
 ...
 '3/01/2013', '4/01/2013', '5/01/2013', '6/01/2013', '7/01/2013',
 '8/01/2013', '9/01/2013', '10/01/2013', '11/01/2013', '12/01/2013'],
 dtype='object', name='date', length=168)

In [7]:
# ADF test to check stationarity
# If p value < 0.05 then stationary else take the differenciation

result = adfuller(data_training.count_.dropna())
print('p-value: %f' % result[1])

p-value: 0.039865

In [8]:
# finding order of AR term using partial autocorrelation plot

fig, axes = plt.subplots(1, 1)
axes.set(ylim=(0,5))
plot_pacf(data_training.count_.diff().dropna(), ax=axes)

display(fig)

In [9]:
# finding order of AR term using autocorrelation plot

fig, axes = plt.subplots(1, 1)
axes.set(ylim=(0,5))
plot_acf(data_training.count_.dropna(), ax=axes)

display(fig)


In [10]:
model = ARIMA(data_training.count_, order=(9,0,3))
results = model.fit(disp=0)
print(results.summary())

/databricks/python/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
 % freq, ValueWarning)
 ARMA Model Results 
==============================================================================
Dep. Variable: count_ No. Observations: 168
Model: ARMA(9, 3) Log Likelihood -1006.317
Method: css-mle S.D. of innovations 93.462
Date: Thu, 12 Dec 2019 AIC 2040.634
Time: 05:13:57 BIC 2084.369
Sample: 01-01-2000 HQIC 2058.384
 - 12-01-2013 
================================================================================
 coef std err z P>|z| [0.025 0.975]
--------------------------------------------------------------------------------
const 570.3173 5665.938 0.101 0.920 -1.05e+04 1.17e+04
ar.L1.count_ 1.6725 0.270 6.189 0.000 1.143 2.202
ar.L2.count_ -0.6551 0.523 -1.252 0.212 -1.680 0.370
ar.L3.count_ -0.4799 0.339 -1.415 0.159 -1.144 0.185
ar.L4.count_ 0.1849 0.172 1.072 0.285 -0.153 0.523
ar.L5.count_ 0.4091 0.179 2.286 0.024 0.058 0.760
ar.L6.count_ -0.1197 0.188 -0.638 0.525 -0.487 0.248
ar.L7.count_ -0.1004 0.176 -0.572 0.568 -0.445 0.244
ar.L8.count_ -0.1062 0.159 -0.667 0.506 -0.418 0.206
ar.L9.count_ 0.1915 0.096 2.004 0.047 0.004 0.379
ma.L1.count_ -1.2441 0.272 -4.577 0.000 -1.777 -0.711
ma.L2.count_ 0.6447 0.401 1.609 0.110 -0.141 1.430
ma.L3.count_ 0.2392 0.267 0.897 0.371 -0.283 0.762
 Roots 
=============================================================================
 Real Imaginary Modulus Frequency
-----------------------------------------------------------------------------
AR.1 -1.2059 -0.5477j 1.3244 -0.4321
AR.2 -1.2059 +0.5477j 1.3244 0.4321
AR.3 -0.6294 -1.2831j 1.4292 -0.3226
AR.4 -0.6294 +1.2831j 1.4292 0.3226
AR.5 0.6576 -0.8707j 1.0911 -0.1471
AR.6 0.6576 +0.8707j 1.0911 0.1471
AR.7 0.9535 -0.5584j 1.1050 -0.0843
AR.8 0.9535 +0.5584j 1.1050 0.0843
AR.9 1.0028 -0.0000j 1.0028 -0.0000
MA.1 0.7419 -0.6707j 1.0002 -0.1170
MA.2 0.7419 +0.6707j 1.0002 0.1170
MA.3 -4.1788 -0.0000j 4.1788 -0.5000
-----------------------------------------------------------------------------

In [11]:
#predictions for the year 2014

pred = results.forecast(12)

In [12]:
fig, ax = plt.subplots(1,1, figsize = (13,5))
plt.plot( data_testing.index, pred[0])
plt.plot( data_testing.index,data_testing.count_)
plt.legend(['pred','actual'])
plt.xlabel('Date')
plt.ylabel('No. of allegations')
plt.title('Predictions on 2014')
display(fig)


In [13]:
print("score for 2014 data",r2_score(data_testing.count_,pred[0] ))

score for 2014 data -0.29008373357349226

In [14]:
# Predictions for year 2015

pred2 = results.forecast(12)
fig, ax = plt.subplots(1,1, figsize = (13,5))
plt.plot( data_predict.index[:11], pred2[0][:11])
plt.plot( data_predict.index[:11],data_predict.count_[:11])
plt.legend(['pred','actual'])
plt.xlabel('Date')
plt.ylabel('No. of allegations')
plt.title('Predictions on 2015')
display(fig)


In [15]:
print("score for 2015 data", r2_score(data_predict.count_[:11],pred2[0][:11]))

score for 2015 data -1.8144730948223144